In [19]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from tqdm import tqdm
import random

In [2]:
data = pd.read_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/mflix-app/static/Movies_data3200.csv')

In [3]:
data

,movieId,imdbId,tmdbId,title,original_language,overview,popularity,budget,genres,production_companies,...,release_date,revenue,runtime,spoken_languages,vote_average,vote_count,keywords,cast,crew,poster_working_url
0,2,113497,8844,Jumanji,en,When siblings Judy and Peter discover an encha...,17.015539,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...",...,1995-12-15,262797249,104,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",6.9,2413,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",https://image.tmdb.org/t/p/w500/vzmL6fP7aPKNKP...
1,20,113845,11517,Money Train,en,A vengeful New York transit cop decides to ste...,7.337906,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...","[{'name': 'Columbia Pictures', 'id': 5}]",...,1995-11-21,35431113,103,"[{'iso_639_1': 'en', 'name': 'English'}]",5.4,224,"[{'id': 380, 'name': 'brother brother relation...","[{'cast_id': 1, 'character': 'John', 'credit_i...","[{'credit_id': '52fe44509251416c750305a1', 'de...",https://image.tmdb.org/t/p/w500/jSozzzVOR2kfXg...
2,34,112431,9598,Babe,en,Babe is a little pig who doesn't quite know hi...,14.404764,30000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'na...","[{'name': 'Universal Pictures', 'id': 33}, {'n...",...,1995-07-18,254134910,89,"[{'iso_639_1': 'en', 'name': 'English'}]",6.0,756,"[{'id': 3020, 'name': 'sheep'}, {'id': 4931, '...","[{'cast_id': 1, 'character': 'Babe the Gallant...","[{'credit_id': '52fe450fc3a36847f80b9ff7', 'de...",https://image.tmdb.org/t/p/w500/gN6X3fwPya8pLf...
3,54,112499,26441,The Big Green,en,"In a depressed Texas town, British foreign exc...",5.982764,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'name': 'Walt Disney Pictures', 'id': 2}, {'...",...,1995-09-29,0,100,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",5.2,41,"[{'id': 6075, 'name': 'sport'}, {'id': 9963, '...","[{'cast_id': 1, 'character': 'Sheriff Tom Palm...","[{'credit_id': '52fe4503c3a368484e04375d', 'de...",https://image.tmdb.org/t/p/w500/erWfSEcjO4LG69...
4,57,113321,9089,Home for the Holidays,en,"After losing her job, making out with her soon...",8.416802,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'name': 'Paramount Pictures', 'id': 4}, {'na...",...,1995-11-02,17519169,103,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",6.3,39,"[{'id': 2672, 'name': 'baltimore'}, {'id': 454...","[{'cast_id': 10, 'character': 'Claudia Larson'...","[{'credit_id': '52fe44dac3a36847f80ae00b', 'de...",https://image.tmdb.org/t/p/w500/97EKLaslQ4gzkv...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,176201,232750,5589,Å Ã­lenÄ› smutnÃ¡ princezna,cs,No overview found.,0.375001,0,"[{'id': 10402, 'name': 'Music'}, {'id': 35, 'n...","[{'name': 'FilmovÃ© Studio Barrandov', 'id': 1...",...,1968-06-07,0,73,"[{'iso_639_1': 'cs', 'name': 'ÄŒeskÃ½'}]",6.1,4,"[{'id': 3205, 'name': 'fairy tale'}, {'id': 13...","[{'cast_id': 12, 'character': 'princezna Helen...","[{'credit_id': '52fe440fc3a36847f807ffa3', 'de...",https://image.tmdb.org/t/p/w500/9h5eegOHh1zoaH...
3196,176215,36975,84710,Jungle Woman,en,Paula the ape woman (Acquanetta) is alive and ...,0.143223,0,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...","[{'name': 'Universal Pictures', 'id': 33}]",...,1944-06-01,0,61,"[{'iso_639_1': 'en', 'name': 'English'}]",5.8,2,"[{'id': 208422, 'name': 'paula dupree'}]","[{'cast_id': 2, 'character': 'Paula Dupree - t...","[{'credit_id': '52fe49099251416c910a0a89', 'de...",https://image.tmdb.org/t/p/w500/udZ3hvhC52su7U...
3197,176235,457437,14885,Pooh's Heffalump Halloween Movie,en,"It's Halloween in the 100 Acre Wood, and Roo's...",2.568495,0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...","[{'name': 'Walt Disney Pictures', 'id': 2}]",...,2005-09-13,0,67,"[{'iso_639_1': 'en', 'name': 'English'}]",5.4,7,"[{'id': 3335, 'name': 'hallo

In [4]:
ratings = pd.read_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/data/ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990


In [5]:
filter_ratings = ratings[ratings['movieId'].isin(data['movieId'])]
filter_ratings

,userId,movieId,rating,timestamp
2,1,858,5.0,1425941523
14,1,58559,4.0,1425942007
45,2,1233,4.0,867039820
52,3,858,4.0,1048076945
75,4,2023,5.0,1042667945
...,...,...,...,...
26024220,270896,6016,4.0,1257034275
26024234,270896,7153,5.0,1257031583
26024257,270896,40815,4.0,1257032001
26024276,270896,52973,3.5,1257034325


In [6]:
filter_ratings.drop(columns = 'timestamp', axis = 1, inplace = True)

/var/folders/qt/lkmkr1817vd45bbs2t4fbrv00000gn/T/ipykernel_2456/725361958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_ratings.drop(columns = 'timestamp', axis = 1, inplace = True)


In [7]:
filter_ratings.to_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/data/filter_ratings_all.dat', sep=':', index=False, header=False)

Above is the userId for some 1000 users and corresponding movieId - For now computing database of only 1000 users.

In [8]:
data_df = pd.read_csv('/Users/saptarshimondal/Documents/1.Course_Materials_MSDS/2nd SEM/CSCE 670 ISR/3.Project/MFlix-RecommenderSystem/src/data/filter_ratings_all.dat', sep=':', names=["UserID", "MovieID", "Rating"], engine='python')

In [9]:
data_df

,UserID,MovieID,Rating
0,1,858,5.0
1,1,58559,4.0
2,2,1233,4.0
3,3,858,4.0
4,4,2023,5.0
...,...,...,...
2044608,270896,6016,4.0
2044609,270896,7153,5.0
2044610,270896,40815,4.0
2044611,270896,52973,3.5


In [10]:
# First, generate dictionaries for mapping old id to new id for users and movies
unique_MovieID = data_df['MovieID'].unique()
unique_UserID = data_df['UserID'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_MovieID:
    movie_old2new_id_dict[i] = j
    j += 1

# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = data_df['UserID'].values
movie_list = data_df['MovieID'].values
for j in range(len(data_df)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
data_df['UserID'] = user_list
data_df['MovieID'] = movie_list

# generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
np.random.seed(12)
train_index = np.random.random(len(data_df)) <= 0.7
train_df = data_df[train_index]
test_df = data_df[~train_index]

# generate train_mat and test_mat
num_user = len(data_df['UserID'].unique())
num_movie = len(data_df['MovieID'].unique())

train_mat = coo_matrix((train_df['Rating'].values, (train_df['UserID'].values, train_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()
test_mat = coo_matrix((test_df['Rating'].values, (test_df['UserID'].values, test_df['MovieID'].values)), shape=(num_user, num_movie)).astype(float).toarray()

In [45]:
data_df

,UserID,MovieID,Rating
0,0,0,5.0
1,0,1,4.0
2,1,2,4.0
3,2,0,4.0
4,3,3,5.0
...,...,...,...
47358,4997,197,4.0
47359,4998,245,4.5
47360,4998,146,3.0
47361,4998,215,1.0


In [11]:
train_mat.shape

(218417, 3145)

In [12]:
test_mat.shape

(218417, 3145)

In [16]:
train_mat

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       ...,
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [5., 5., 0., ..., 0., 0., 0.]])

In [20]:
class MF_withBias:
    def __init__(self, train_mat, test_mat, latent=5, lr=0.01, reg=0.01):
        self.train_mat = train_mat  
        self.test_mat = test_mat  

        self.latent = latent  
        self.lr = lr 
        self.reg = reg 

        self.num_user, self.num_movie = train_mat.shape

        self.sample_user, self.sample_movie = self.train_mat.nonzero()  # get the user-movie paris having ratings in train_mat
        self.num_sample = len(self.sample_user)  # the number of user-movie pairs having ratings in train_mat

        self.train_indicator_mat = 1.0 * (train_mat > 0)  # binary matrix to indicate whether s user-movie pair has rating or not in train_mat
        self.test_indicator_mat = 1.0 * (test_mat > 0)  # binary matrix to indicate whether s user-movie pair has rating or not in test_mat

        self.P = np.random.random((self.num_user, self.latent))  # latent factors for users, size (#user, self.latent), randomly initialized
        self.Q = np.random.random((self.num_movie, self.latent))  # latent factors for users, size (#movie, self.latent), randomly initialized
        self.user_bias = np.random.random(self.num_user)
        self.movie_bias = np.random.random(self.num_movie)

    def train(self, epoch=20, verbose=True):
       
        epoch_loss_list = []
        epoch_test_RMSE_list = []
        indices = list(range(self.num_sample))
        for ep in tqdm(range(epoch)):
         
            random.shuffle(indices)
            total_loss = 0.0
            for index in indices:
                u = self.sample_user[index]
                i = self.sample_movie[index]

                rating = self.train_mat[u,i]
                prediction = np.dot(self.P[u], self.Q[i].T) + self.user_bias[u] + self.movie_bias[i]

                error = prediction - rating
                user_learn = 2 * error * self.Q[i] + 2 * self.reg * self.P[u]
                movie_learn = 2 * error * self.P[u] + 2 * self.reg * self.Q[i]
                user_bias_learn = 2 * error + 2 * self.reg * self.user_bias[u]   
                movie_bias_learn = 2 * error + 2 * self.reg * self.movie_bias[i] #Adding Bias terms

                self.P[u] = self.P[u] - self.lr * user_learn
                self.Q[i] = self.Q[i] - self.lr * movie_learn
                self.user_bias[u] = self.user_bias[u] - self.lr * user_bias_learn
                self.movie_bias[i] = self.movie_bias[i] - self.lr *  movie_bias_learn

                total_loss += error**2

            
            prediction_mat = np.matmul(self.P, self.Q.T) + self.user_bias[:, np.newaxis] + self.movie_bias[np.newaxis, :]
            test_mat_nan = np.where(self.test_mat == 0, np.nan, self.test_mat)
            rmse = np.sqrt(np.nanmean((prediction_mat - test_mat_nan)**2))
            avg_train_loss = total_loss / self.num_sample
            epoch_loss_list.append(avg_train_loss)
            epoch_test_RMSE_list.append(rmse)

            if verbose:
                print(f"Epoch {ep+1}: Average Training Loss = {avg_train_loss} | Test RMSE = {rmse}")

        return epoch_loss_list, epoch_test_RMSE_list


    def predict(self):
        prediction_mat = np.matmul(self.P, self.Q.T) + self.user_bias[:, np.newaxis] + self.movie_bias[np.newaxis, :]
        return prediction_mat

In [21]:
mf = MF_withBias(train_mat, test_mat)
epoch_loss_list, epoch_test_RMSE_list = mf.train(epoch=20, verbose = False)

100%|██████████| 20/20 [10:48<00:00, 32.42s/it]


In [23]:
preds = mf.predict()

In [25]:
type(preds)

numpy.ndarray

In [31]:
movie_id = np.argsort(np.mean(preds, axis =0))

In [34]:
movie_id[::-1][:5]

array([1726, 1732, 1310,  983, 1730])